# Part 5

## Import & Load

In [ ]:
%load_ext google.cloud.bigquery

In [ ]:
import os
import pandas as pd
from google.cloud import bigquery
import google.auth
from google.cloud import bigquery
import pandas_gbq
from datetime import date, timedelta
import time
# import math

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'C:/Users/DIKA/Documents/bootcamp/Untitled Folder/aqueous-freedom-398208-cfe0d4e567f2.json'

In [ ]:
bigquery_client = bigquery.Client()

## Segmentasi (Done)

#### Dataset Tim 2

In [ ]:
query = """
DROP TABLE IF EXISTS `aqueous-freedom-398208.dkf_summary_tim2.summary_segmentasi`;
CREATE TABLE `aqueous-freedom-398208.dkf_summary_tim2.summary_segmentasi` AS
    WITH sub AS (
        SELECT sub2.cust_name, COUNT(*) AS total_transaction, SUM(sub2.price*sub2.quantity) AS total_spending
        FROM (
            SELECT c.cust_name, o.order_date, f.price, o.quantity
            FROM `aqueous-freedom-398208.dkf_okefood.order` o
            INNER JOIN `aqueous-freedom-398208.dkf_okefood.customer` c ON c.cust_id = o.fk_cust_id
            INNER JOIN `aqueous-freedom-398208.dkf_okefood.food` f ON f.food_id = o.fk_food_id
            WHERE o.order_date > CURRENT_DATE()-30 AND o.order_date <= CURRENT_DATE()
            ) AS sub2
        GROUP BY 1
    )

    SELECT sub.cust_name AS name, sub.total_transaction, sub.total_spending,
    CASE
        WHEN sub.total_spending > 500000 THEN 'Big Spender'
        WHEN sub.total_transaction < 2 THEN 'New'
        WHEN sub.total_transaction < 6 THEN 'Good'
        WHEN sub.total_transaction < 11 THEN 'Potential'
        ELSE 'Loyalist'
    END segmentation
    FROM sub
    ORDER BY 1;
  """
result = bigquery_client.query(query)
# df = result.to_dataframe()
# df

#### Dataset Coding.ID

In [ ]:
query = """
DROP TABLE IF EXISTS `aqueous-freedom-398208.dkf_summary_codingid.summary_segmentasi`;
CREATE TABLE `aqueous-freedom-398208.dkf_summary_codingid.summary_segmentasi` AS

    WITH sub AS (
        SELECT sub2.name, COUNT(*) AS total_transaction, SUM(sub2.price) AS total_spending
        FROM (
            SELECT c.name, t.transaction_date, t.price
            FROM `aqueous-freedom-398208.ok_food_dataset.transaction_2024` t
            INNER JOIN `aqueous-freedom-398208.ok_food_dataset.customer` c ON c.id = t.customer_id
            WHERE t.transaction_date > CURRENT_DATE()-30 AND t.transaction_date <= CURRENT_DATE()
            ) AS sub2
        GROUP BY 1
    )

    SELECT sub.name, sub.total_transaction, sub.total_spending,
    CASE
        WHEN sub.total_spending > 500000 THEN 'Big Spender'
        WHEN sub.total_transaction < 2 THEN 'New'
        WHEN sub.total_transaction < 6 THEN 'Good'
        WHEN sub.total_transaction < 11 THEN 'Potential'
        ELSE 'Loyalist'
    END segmentation
    FROM sub
    ORDER BY 1;
  """
result = bigquery_client.query(query)
# df = result.to_dataframe()
# df

## Growth (Done)

#### Dataset Tim 2 New

In [ ]:
query = """
    INSERT INTO `aqueous-freedom-398208.dkf_summary_tim2.summary_growth`

    WITH sub AS (
      SELECT r.city AS kota, DATE(o.order_date) AS transaction_date, COALESCE(SUM(f.price * o.quantity), NULL) AS daily_revenue
      FROM `aqueous-freedom-398208.dkf_okefood.order` o
      INNER JOIN `aqueous-freedom-398208.dkf_okefood.food` f ON f.food_id = o.fk_food_id
      INNER JOIN `aqueous-freedom-398208.dkf_okefood.resto` r ON r.resto_id = f.fk_resto_id
      GROUP BY 1, 2
    )
    ,
    skeleton AS (
      SELECT DISTINCT kota, CURRENT_DATE() AS transaction_date
      FROM sub
    )
    ,
    result AS (
      SELECT s.kota, s.transaction_date, COALESCE(sub.daily_revenue, NULL) AS daily_revenue
      FROM skeleton s
      LEFT JOIN sub ON s.kota = sub.kota AND s.transaction_date = sub.transaction_date
    )
    ,
    compare AS (
      SELECT r.kota, r.transaction_date, r.daily_revenue
      FROM result r
      UNION ALL (SELECT kota, transaction_date, daily_revenue FROM `aqueous-freedom-398208.dkf_summary_tim2.summary_growth`)
    )
    ,
    fill AS (
      SELECT c.kota, c.transaction_date, COALESCE(c.daily_revenue, LAG(c.daily_revenue) OVER(PARTITION BY c.kota ORDER BY c.transaction_date)) AS daily_revenue FROM compare c
    )
    ,
    growth AS (
        SELECT f.kota, f.transaction_date, f.daily_revenue,
        ROUND((f.daily_revenue - LAG(f.daily_revenue) OVER(PARTITION BY f.kota ORDER BY f.transaction_date)) / NULLIF(LAG(f.daily_revenue) OVER(PARTITION BY f.kota ORDER BY f.transaction_date), 0) * 100, 2) AS growth
        FROM fill f
    )

    SELECT g.kota, g.transaction_date, g.daily_revenue, g.growth
    FROM growth g
    WHERE g.transaction_date = CURRENT_DATE()
    ORDER BY 1,2
"""
result = bigquery_client.query(query)

#### Dataset Codingid New

In [ ]:
query = """
    INSERT INTO `aqueous-freedom-398208.dkf_summary_codingid.summary_growth`

    WITH sub AS (
      SELECT r.kota, DATE(t.transaction_date) AS transaction_date, COALESCE(SUM(t.price), NULL) AS daily_revenue
      FROM `aqueous-freedom-398208.ok_food_dataset.transaction_2024` t
      INNER JOIN `aqueous-freedom-398208.ok_food_dataset.resto` r ON r.id = t.resto_id
      GROUP BY 1, 2
    )
    ,
    skeleton AS (
      SELECT DISTINCT kota, CURRENT_DATE() AS transaction_date
      FROM sub
    )
    ,
    result AS (
      SELECT s.kota, s.transaction_date, COALESCE(sub.daily_revenue, NULL) AS daily_revenue
      FROM skeleton s
      LEFT JOIN sub ON s.kota = sub.kota AND s.transaction_date = sub.transaction_date
    )
    ,
    compare AS (
      SELECT r.kota, r.transaction_date, r.daily_revenue
      FROM result r
      UNION ALL (SELECT kota, transaction_date, daily_revenue FROM `aqueous-freedom-398208.dkf_summary_codingid.summary_growth`)
    )
    ,
    fill AS (
      SELECT c.kota, c.transaction_date, COALESCE(c.daily_revenue, LAG(c.daily_revenue) OVER(PARTITION BY c.kota ORDER BY c.transaction_date)) AS daily_revenue FROM compare c
    )
    ,
    growth AS (
        SELECT f.kota, f.transaction_date, f.daily_revenue,
        ROUND((f.daily_revenue - LAG(f.daily_revenue) OVER(PARTITION BY f.kota ORDER BY f.transaction_date)) / NULLIF(LAG(f.daily_revenue) OVER(PARTITION BY f.kota ORDER BY f.transaction_date), 0) * 100, 2) AS growth
        FROM fill f
    )

    SELECT g.kota, g.transaction_date, COALESCE(g.daily_revenue, 0), g.growth
    FROM growth g
    WHERE g.transaction_date = CURRENT_DATE()
    ORDER BY 1,2
"""
result = bigquery_client.query(query)

#### Dataset Tim 2

In [ ]:
# dmin = 30
for i in range(10):
# query = f"""
#     WITH sub AS (
#       SELECT sub2.kota, sub2.transaction_date, SUM(sub2.price*sub2.quantity) AS daily_revenue
#       FROM (
#         SELECT r.city AS kota, o.order_date AS transaction_date, f.price, o.quantity
#         FROM `aqueous-freedom-398208.dkf_okefood.order` o
#         INNER JOIN `aqueous-freedom-398208.dkf_okefood.food` f ON f.food_id = o.fk_food_id
#         INNER JOIN `aqueous-freedom-398208.dkf_okefood.resto` r ON r.resto_id = f.fk_resto_id
#         WHERE o.order_date > CURRENT_DATE()-{dmin} AND o.order_date <= CURRENT_DATE()
#         ) AS sub2
#       GROUP BY 1, 2
#     )

#     SELECT sub.kota, sub.transaction_date, sub.daily_revenue,
#     FROM sub
#     ORDER BY 1, 2
#   """

# result = bigquery_client.query(query)
# df = result.to_dataframe()
# df['transaction_date'] = pd.to_datetime(df['transaction_date'])

# result = df.iloc[:0,:].copy()
# kota = df.kota.unique()

# for i in kota:
#     for j in range(dmin):
#         day = date.today() - timedelta(days = j)
#         day = pd.to_datetime(day)
#         # if df3['transaction_date'][k] == day:
#         result = result._append({'kota':i, 'transaction_date':day, 'daily_revenue':0}, ignore_index=True)

# result = result.sort_values(by=['kota', 'transaction_date'], ignore_index=True)

# for i in range(len(df)):
#     result.loc[(result['kota'] == df['kota'][i]) & (result['transaction_date'] == df['transaction_date'][i])] = [df['kota'][i],df['transaction_date'][i],df['daily_revenue'][i]]

# result.insert(3, 'original_revenue', result['daily_revenue'])

# for i in range(1,len(result)):
#     if result['kota'][i] == result['kota'][i-1]:
#         if result['daily_revenue'][i] == 0:
#             result['daily_revenue'][i] = result['daily_revenue'][i-1]

# # print(backup.to_markdown())

# growth = []
# for h in range(len(kota)):
#     for i in range(1,dmin):
#         if i == 1:
#             growth.append(float('nan'))
#         if result['daily_revenue'][(h*dmin)+i] == 0 or result['daily_revenue'][(h*dmin)+i-1] == 0:
#             growth.append(0)
#         else:
#             x = round((result['daily_revenue'][(h*dmin)+i]-result['daily_revenue'][(h*dmin)+i-1])/result['daily_revenue'][(h*dmin)+i-1]*100, 2)
#             growth.append(x)

# result.insert(3, 'growth', growth)
# # print(result.to_markdown())

# result['transaction_date'] = pd.to_datetime(result['transaction_date'])

# result.rename(columns = {'kota':'city'}, inplace = True)

# pandas_gbq.to_gbq(result, 'dkf_summary_tim2.summary_growth', project_id='aqueous-freedom-398208', if_exists='replace')

100%|██████████| 1/1 [00:00<?, ?it/s]


#### Dataset Coding.ID

In [ ]:
# dmin = 30

# query = f"""
#     WITH sub AS (
#       SELECT sub2.kota, sub2.transaction_date, SUM(sub2.price) AS daily_revenue
#       FROM (
#         SELECT r.kota, t.transaction_date, t.price
#         FROM `aqueous-freedom-398208.ok_food_dataset.transaction_2024` t
#         INNER JOIN `aqueous-freedom-398208.ok_food_dataset.resto` r ON r.id = t.resto_id
#         WHERE t.transaction_date > CURRENT_DATE()-{dmin} AND t.transaction_date <= CURRENT_DATE()
#         ) AS sub2
#       GROUP BY 1, 2
#     )

#     SELECT sub.kota, sub.transaction_date, sub.daily_revenue,
#     FROM sub
#     ORDER BY 1, 2
#   """

# result = bigquery_client.query(query)
# df = result.to_dataframe()
# df['transaction_date'] = pd.to_datetime(df['transaction_date'])

# result = df.iloc[:0,:].copy()
# kota = df.kota.unique()

# for i in kota:
#     for j in range(dmin):
#         day = date.today() - timedelta(days = j)
#         day = pd.to_datetime(day)
#         # if df3['transaction_date'][k] == day:
#         result = result._append({'kota':i, 'transaction_date':day, 'daily_revenue':0}, ignore_index=True)

# result = result.sort_values(by=['kota', 'transaction_date'], ignore_index=True)

# for i in range(len(df)):
#     result.loc[(result['kota'] == df['kota'][i]) & (result['transaction_date'] == df['transaction_date'][i])] = [df['kota'][i],df['transaction_date'][i],df['daily_revenue'][i]]

# result.insert(3, 'original_revenue', result['daily_revenue'])

# for i in range(1,len(result)):
#     if result['kota'][i] == result['kota'][i-1]:
#         if result['daily_revenue'][i] == 0:
#             result['daily_revenue'][i] = result['daily_revenue'][i-1]

# # print(backup.to_markdown())

# growth = []
# for h in range(len(kota)):
#     for i in range(1,dmin):
#         if i == 1:
#             growth.append(float('nan'))
#         if result['daily_revenue'][(h*dmin)+i] == 0 or result['daily_revenue'][(h*dmin)+i-1] == 0:
#             growth.append(0)
#         else:
#             x = round((result['daily_revenue'][(h*dmin)+i]-result['daily_revenue'][(h*dmin)+i-1])/result['daily_revenue'][(h*dmin)+i-1]*100, 2)
#             growth.append(x)

# result.insert(3, 'growth', growth)
# # print(result.to_markdown())

# result['transaction_date'] = pd.to_datetime(result['transaction_date'])

# pandas_gbq.to_gbq(result, 'dkf_summary_codingid.summary_growth', project_id='aqueous-freedom-398208', if_exists='replace')
# # result

100%|██████████| 1/1 [00:00<?, ?it/s]


#### Simulasi Daily Schedule Dataset Tim 2 New

In [ ]:
# query = """
# SELECT DATE_SUB(CURRENT_DATE(), INTERVAL OFFSET DAY) AS transaction_date FROM UNNEST(GENERATE_ARRAY(0, 29)) AS OFFSET ORDER BY 1;
#   """
# result = bigquery_client.query(query)
# df = result.to_dataframe()

# for i in df['transaction_date']:
#     # print(i)
#     query = f"""
#         INSERT INTO `aqueous-freedom-398208.dkf_summary_tim2.summary_growth`

#         WITH sub AS (
#           SELECT r.city AS kota, DATE(o.order_date) AS transaction_date, COALESCE(SUM(f.price * o.quantity), NULL) AS daily_revenue
#           FROM `aqueous-freedom-398208.dkf_okefood.order` o
#           INNER JOIN `aqueous-freedom-398208.dkf_okefood.food` f ON f.food_id = o.fk_food_id
#           INNER JOIN `aqueous-freedom-398208.dkf_okefood.resto` r ON r.resto_id = f.fk_resto_id
#           GROUP BY 1, 2
#         )
#         ,
#         skeleton AS (
#           SELECT DISTINCT kota, CAST('{i}' AS DATE) AS transaction_date
#           FROM sub
#         )
#         ,
#         result AS (
#           SELECT s.kota, s.transaction_date, COALESCE(sub.daily_revenue, NULL) AS daily_revenue
#           FROM skeleton s
#           LEFT JOIN sub ON s.kota = sub.kota AND s.transaction_date = sub.transaction_date
#         )
#         ,
#         compare AS (
#           SELECT r.kota, r.transaction_date, r.daily_revenue
#           FROM result r
#           UNION ALL (SELECT kota, transaction_date, daily_revenue FROM `aqueous-freedom-398208.dkf_summary_tim2.summary_growth`)
#         )
#         ,
#         fill AS (
#           SELECT c.kota, c.transaction_date, COALESCE(c.daily_revenue, LAG(c.daily_revenue) OVER(PARTITION BY c.kota ORDER BY c.transaction_date)) AS daily_revenue FROM compare c
#         )
#         ,
#         growth AS (
#             SELECT f.kota, f.transaction_date, f.daily_revenue,
#             ROUND((f.daily_revenue - LAG(f.daily_revenue) OVER(PARTITION BY f.kota ORDER BY f.transaction_date)) / NULLIF(LAG(f.daily_revenue) OVER(PARTITION BY f.kota ORDER BY f.transaction_date), 0) * 100, 2) AS growth
#             FROM fill f
#         )

#         SELECT g.kota, g.transaction_date, COALESCE(g.daily_revenue, 0), g.growth
#         FROM growth g
#         WHERE g.transaction_date = '{i}'
#         ORDER BY 1,2
#     """
#     time.sleep(2)
#     result = bigquery_client.query(query)




#### Simulasi Daily Schedule Dataset Codingid

In [ ]:
# query = """
# SELECT DATE_SUB(CURRENT_DATE(), INTERVAL OFFSET DAY) AS transaction_date FROM UNNEST(GENERATE_ARRAY(0, 29)) AS OFFSET ORDER BY 1;
#   """
# result = bigquery_client.query(query)
# df = result.to_dataframe()

# for i in df['transaction_date']:
#     # print(i)
#     query = f"""
#         INSERT INTO `aqueous-freedom-398208.dkf_summary_codingid.summary_growth`

#         WITH sub AS (
#           SELECT r.kota, DATE(t.transaction_date) AS transaction_date, COALESCE(SUM(t.price), NULL) AS daily_revenue
#           FROM `aqueous-freedom-398208.ok_food_dataset.transaction_2024` t
#           INNER JOIN `aqueous-freedom-398208.ok_food_dataset.resto` r ON r.id = t.resto_id
#           GROUP BY 1, 2
#         )
#         ,
#         skeleton AS (
#           SELECT DISTINCT kota, CAST('{i}' AS DATE) AS transaction_date
#           FROM sub
#         )
#         ,
#         result AS (
#           SELECT s.kota, s.transaction_date, COALESCE(sub.daily_revenue, NULL) AS daily_revenue
#           FROM skeleton s
#           LEFT JOIN sub ON s.kota = sub.kota AND s.transaction_date = sub.transaction_date
#         )
#         ,
#         compare AS (
#           SELECT r.kota, r.transaction_date, r.daily_revenue
#           FROM result r
#           UNION ALL (SELECT kota, transaction_date, daily_revenue FROM `aqueous-freedom-398208.dkf_summary_codingid.summary_growth`)
#         )
#         ,
#         fill AS (
#           SELECT c.kota, c.transaction_date, COALESCE(c.daily_revenue, LAG(c.daily_revenue) OVER(PARTITION BY c.kota ORDER BY c.transaction_date)) AS daily_revenue FROM compare c
#         )
#         ,
#         growth AS (
#             SELECT f.kota, f.transaction_date, f.daily_revenue,
#             ROUND((f.daily_revenue - LAG(f.daily_revenue) OVER(PARTITION BY f.kota ORDER BY f.transaction_date)) / NULLIF(LAG(f.daily_revenue) OVER(PARTITION BY f.kota ORDER BY f.transaction_date), 0) * 100, 2) AS growth
#             FROM fill f
#         )

#         SELECT g.kota, g.transaction_date, COALESCE(g.daily_revenue, 0), g.growth
#         FROM growth g
#         WHERE g.transaction_date = '{i}'
#         ORDER BY 1,2
#     """
#     time.sleep(2)
#     result = bigquery_client.query(query)

## Kontribusi (Done)

#### Dataset Tim 2

In [ ]:
query = """
DROP TABLE IF EXISTS `aqueous-freedom-398208.dkf_summary_tim2.summary_kontribusi`;
CREATE TABLE `aqueous-freedom-398208.dkf_summary_tim2.summary_kontribusi` AS
    WITH daily_revenue AS (
        SELECT DATE(o.order_date) AS transaction_date, r.city, SUM(o.quantity * f.price) AS city_revenue
        FROM `aqueous-freedom-398208.dkf_okefood.order` o
        INNER JOIN `aqueous-freedom-398208.dkf_okefood.food` f ON f.food_id = o.fk_food_id
        INNER JOIN `aqueous-freedom-398208.dkf_okefood.customer` c ON c.cust_id = o.fk_cust_id
        INNER JOIN `aqueous-freedom-398208.dkf_okefood.resto` r ON r.resto_id = f.fk_resto_id
        WHERE o.order_date > CURRENT_DATE()-30 AND o.order_date <= CURRENT_DATE()
        GROUP BY DATE(o.order_date), r.city
    ),
    total_daily_revenue AS (
        SELECT DATE(o.order_date)AS transaction_date, SUM(f.price*o.quantity) AS total_revenue,
        FROM `aqueous-freedom-398208.dkf_okefood.order` o
        INNER JOIN `aqueous-freedom-398208.dkf_okefood.food` f ON f.food_id = o.fk_food_id
        INNER JOIN `aqueous-freedom-398208.dkf_okefood.customer` c ON c.cust_id = o.fk_cust_id
        INNER JOIN `aqueous-freedom-398208.dkf_okefood.resto` r ON r.resto_id = f.fk_resto_id
        WHERE o.order_date > CURRENT_DATE()-30 AND o.order_date <= CURRENT_DATE()
        GROUP BY 1
        ORDER BY 1
    )
    SELECT dr.transaction_date, dr.city AS kota, dr.city_revenue, tdr.total_revenue, ROUND((dr.city_revenue / tdr.total_revenue) * 100) AS contribution_percent
    FROM daily_revenue dr
    JOIN total_daily_revenue tdr ON dr.transaction_date = tdr.transaction_date
    ORDER BY dr.transaction_date, dr.city;
  """
result = bigquery_client.query(query)
# df = result.to_dataframe()
# df

#### Dataset Coding.ID

In [ ]:
query = """
DROP TABLE IF EXISTS `aqueous-freedom-398208.dkf_summary_codingid.summary_kontribusi`;
CREATE TABLE `aqueous-freedom-398208.dkf_summary_codingid.summary_kontribusi` AS
    WITH daily_revenue AS (
        SELECT r.kota, t.transaction_date, SUM(t.price) AS city_revenue,
        FROM `aqueous-freedom-398208.ok_food_dataset.transaction_2024` t
        INNER JOIN `aqueous-freedom-398208.ok_food_dataset.customer` c ON c.id = t.customer_id
        INNER JOIN `aqueous-freedom-398208.ok_food_dataset.resto` r ON r.id = t.resto_id
        WHERE t.transaction_date > CURRENT_DATE()-30 AND t.transaction_date <= CURRENT_DATE()
        GROUP BY 1, 2
    ),
    total_daily_revenue AS (
        SELECT t.transaction_date, SUM(t.price) AS total_revenue,
        FROM `aqueous-freedom-398208.ok_food_dataset.transaction_2024` t
        INNER JOIN `aqueous-freedom-398208.ok_food_dataset.customer` c ON c.id = t.customer_id
        INNER JOIN `aqueous-freedom-398208.ok_food_dataset.resto` r ON r.id = t.resto_id
        WHERE t.transaction_date > CURRENT_DATE()-30 AND t.transaction_date <= CURRENT_DATE()
        GROUP BY 1
    )
    SELECT
        dr.transaction_date,
        dr.kota,
        dr.city_revenue,
        tdr.total_revenue,
       ROUND((dr.city_revenue / tdr.total_revenue) * 100) AS contribution_percent
    FROM
        daily_revenue dr
    JOIN
        total_daily_revenue tdr ON dr.transaction_date = tdr.transaction_date
    ORDER BY
        dr.transaction_date, dr.kota;
  """
result = bigquery_client.query(query)
# df = result.to_dataframe()
# df